In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
# import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
# translator = str.maketrans('', '', string.punctuation)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/reinier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/reinier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Load EmoLex
emolex_df = pd.read_csv('data/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', sep='\t', names=['word', 'emotion','association'])
emolex_df = emolex_df.dropna()
emolex_df = emolex_df[emolex_df.association == 1]
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association')
emolex_words = emolex_words.reset_index()


In [5]:
# emotions = emolex_df['emotion'].unique()
# emolex_words = emolex_df.pivot(index='word',
#                                columns='emotion')
# emolex_words.head()

In [8]:
path = "data/subset_2000.csv"
reviews_df = pd.read_csv(path,sep=',')
reviews_df.head()

,Unnamed: 0,review_id,text,stars,positive,neutral,negative,compound_sentiment,newstar,anger,anticipation,disgust,fear,joy,negative.1,positive.1,sadness,surprise,trust,word_count
0,0,fj7N9Lp6AvEEy6LHrDZzjw,When I was shown to my seat of was still wet s...,2,0.036,0.846,0.118,-0.6437,1,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,26.0
1,1,YX2cFHDxlUfGnQ8bHPq4cA,Not Impressed at all. Ordered a omelette and b...,1,0.078,0.715,0.207,-0.8337,1,0.0,1.0,1.0,1.0,1.0,3.0,2.0,3.0,0.0,1.0,33.0
2,2,pDN3hRBarmGWXbK64A83MA,never coming back here again. all of the glass...,1,0.065,0.865,0.070,-0.3773,1,0.0,1.0,3.0,1.0,1.0,7.0,2.0,2.0,0.0,1.0,64.0
3,3,ae5On6KCPiglMQJ--1JcTQ,I don't recommend this place for breakfast. Th...,2,0.000,0.860,0.140,-0.7672,1,0.0,1.0,0.0,0.0,3.0,3.0,6.0,0.0,1.0,3.0,23.0
4,4,i5jMeyoJSbUrQ7T-AU22_A,"Well, lots to say. Managers were busy makin co...",1,0.124,0.741,0.134,-0.3129,1,2.0,4.0,2.0,1.0,1.0,4.0,3.0,1.0,0.0,1.0,60.0


In [9]:
def count_emotions(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator).lower()
    words = word_tokenize(text)
    emotions_count = emolex_words[emolex_words.word.isin(words)].sum()
    emotions_count['word_count'] = len(words)

    return emotions_count

    # words = [word for word in words if word.lower() not in stop_words]
    # return words

In [10]:
documents = ['Text good bad scary danger  1', 'yummy nice !!!disgusting hate', 'This Restaurant was amazing! bad and good test for yummy']
emotions_count = [count_emotions(doc) for doc in documents]

In [11]:
df = pd.DataFrame(emotions_count)
df.index = documents

print(df)

emotion                                                       word  anger  \
Text good bad scary danger  1                        baddangergood    1.0   
yummy nice !!!disgusting hate                       disgustinghate    2.0   
This Restaurant was amazing! bad and good test ...         badgood    1.0   

emotion                                             anticipation  disgust  \
Text good bad scary danger  1                                1.0      1.0   
yummy nice !!!disgusting hate                                0.0      2.0   
This Restaurant was amazing! bad and good test ...           1.0      1.0   

emotion                                             fear  joy  negative  \
Text good bad scary danger  1                        2.0  1.0       2.0   
yummy nice !!!disgusting hate                        2.0  0.0       2.0   
This Restaurant was amazing! bad and good test ...   1.0  1.0       1.0   

emotion                                             positive  sadness  \
Text goo

In [7]:
text = "This Restaurant was amazing! bad and good test for yummy"
clean_text = remove_unnecessary(text)
clean_text

['restaurant', 'amazing', 'bad', 'good', 'test', 'yummy']

In [14]:
results = pd.DataFrame(columns= ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative',
                                 'positive', 'sadness', 'surprise', 'trust'])

for word in clean_text:
    if word in emolex_words:
        results.append(word)

results

[]